In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys

nb_path = '/content/drive/MyDrive/Colab Notebooks/site-packages'
sys.path.insert(0, nb_path)  # or append(nb_path)

In [3]:
# Standard library imports
import numpy as np

from gluonts.dataset.common import ListDataset
from gluonts.model.estimator import GluonEstimator
from gluonts.model.predictor import Predictor
from gluonts.trainer import Trainer

import mxnet as mx
from mxnet import gluon
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: gluonts.trainer is deprecated. Use gluonts.mx.trainer instead.
  import sys


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

In [5]:
mx.random.seed(0)
np.random.seed(0)

## **deepAR**

In [169]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ppikppik/calm_hyundai_car.csv", encoding='EUC-KR')

In [170]:
year_start = 1985
month = 12
day = 1
cnt_year = 0
for i in df.index :
  df.loc[i,'회계년도'] = pd.Timestamp(year=year_start+cnt_year, month=12, day=1)
  cnt_year = cnt_year + 1

In [172]:
temp = df.set_index(df['회계년도'])
temp[{ '매출총이익(요약)','당기순이익(요약)','자본총계(요약)','부채총계(요약)','영업이익_차분'}]

,부채총계(요약),영업이익_차분,매출총이익(요약),자본총계(요약),당기순이익(요약)
회계년도,,,,,
1985-12-01,272348,8339,36021,52840,2452
1986-12-01,346239,2394,39006,55381,2523
1987-12-01,360890,4368,41514,55925,7068
1988-12-01,486974,26756,75470,183125,14289
1989-12-01,662586,15324,97313,229293,17414
1990-12-01,1088274,-10,104746,229490,10219
1991-12-01,1229340,29777,143773,232680,5550
1992-12-01,1392061,-78448,71118,238280,-48947
1993-12-01,1467188,80907,158047,287286,18558


In [173]:
def find_upper_lower_bound(df,col):
    temp = sorted(df[col])
    q1, q3 = np.percentile(temp, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    return lower_bound, upper_bound

def range_check_correct(df,col):
    cor_range = find_upper_lower_bound(df,col)
    print(cor_range)
    for i in df.index:
        if df.loc[i][col] < (float)(cor_range[0]) :
            df[col][i] = (float)(cor_range[0])
        if df.loc[i][col] > (float)(cor_range[1]) :
            df[col][i] = (float)(cor_range[1])
    return df

In [174]:
temp = range_check_correct(temp,'영업이익_차분')

(-203587.125, 348615.875)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [175]:
temp

,회사명,연도,거래소코드,회계년도,자산총계(요약),부채총계(요약),자본총계(요약),매출액(요약),매출원가(요약),매출총이익(요약),판매비와 관리비(요약),영업이익(요약),당기순이익(요약),유동자산(요약),유동부채(요약),비유동부채(요약),자본금(요약),비유동자산(요약),총자산수익률(ROA),자기자본수익률(ROE),영업이익률,당기순이익률,유동비율,자기자본비율,부채비용,비유동비율,비유동장기적합률,영업이익_차분
회계년도,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1985-12-01,현대모비스(주),1986,12330,1985-12-01 00:00:00,325188,272348,52840,526262,490241,36021,25438,10583,2452,178331,197264,75084,41153,146857,0.007540,0.046404,0.020110,0.004659,0.904022,0.162491,6.617938,0.511273,1.263427,8339
1986-12-01,현대모비스(주),1987,12330,1986-12-01 00:00:00,401620,346239,55381,591277,552270,39006,26029,12977,2523,223211,251126,95113,41153,178408,0.006282,0.045557,0.021947,0.004267,0.888841,0.137894,8.413457,0.533121,1.309263,2394
1987-12-01,현대모비스(주),1988,12330,1987-12-01 00:00:00,416814,360890,55925,567256,525742,41514,24169,17345,7068,226896,282271,76276,41153,189918,0.016957,0.126384,0.030577,0.012460,0.803823,0.134172,8.712536,0.401626,1.617301,4368
1988-12-01,현대모비스(주),1989,12330,1988-12-01 00:00:00,670100,486974,183125,676821,601351,75470,31368,44101,14289,316071,406551,78666,70000,354028,0.021324,0.078029,0.065159,0.021112,0.777445,0.273281,6.931671,0.222203,2.381365,26756
1989-12-01,현대모비스(주),1990,12330,1989-12-01 00:00:00,891879,662586,229293,859123,761810,97313,37888,59425,17414,376945,474367,187048,80000,514934,0.019525,0.075946,0.069169,0.020270,0.794627,0.257090,8.267687,0.363247,1.928245,15324
1990-12-01,현대모비스(주),1991,12330,1990-12-01 00:00:00,1317763,1088274,229490,1007266,902520,104746,45331,59415,10219,573950,704190,383498,80000,743813,0.007755,0.044529,0.058986,0.010145,0.815050,0.174151,13.596100,0.515584,1.604781,-10
1991-12-01,현대모비스(주),1992,12330,1991-12-01 00:00:00,1462020,1229340,232680,1264054,1120281,143773,54581,89192,5550,739637,909476,319865,82400,722383,0.003796,0.023853,0.070560,0.004391,0.813256,0.159150,14.919187,0.442791,1.795789,29777
1992-12-01,현대모비스(주),1993,12330,1992-12-01 00:00:00,1630341,1392061,238280,1343745,1272627,71118,60374,10744,-48947,836148,1069464,322597,106472,794193,-0.030023,-0.205418,0.007996,-0.036426,0.781838,0.146153,13.074433,0.406195,1.850968,-78448
1993-12-01,현대모비스(주),1994,12330,1993-12-01 00:00:00,1754473,1467188,287286,1584369,1426321,158047,66396,91651,18558,919497,957450,509738,106778,834976,0.010578,0.064598,0.057847,0.011713,0.960360,0.163745,13.740546,0.610482,1.354346,80907


In [176]:
prediction_window = 1

In [177]:
train_ds = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익_차분'].values[:-prediction_window] ,
      "feat_dynamic_real": [temp['매출총이익(요약)'].values[:-prediction_window],temp['당기순이익(요약)'].values[:-prediction_window],temp['자산총계(요약)'].values[:-prediction_window],temp['부채총계(요약)'].values[:-prediction_window]]}],
                        freq = "365D"
)

In [178]:
freq = "365D"

estimator = DeepAREstimator(prediction_length = prediction_window, use_feat_dynamic_real= True , freq=freq, cell_type = 'lstm', 
                           trainer=Trainer(epochs=200, ctx = mx.context.gpu(),))

In [179]:
predictor = estimator.train(training_data = train_ds)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:00<00:00, 76.68it/s, epoch=102/200, avg_epoch_loss=11.4]


In [180]:
from gluonts.evaluation.backtest import make_evaluation_predictions

test_data = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익_차분'].values,
      "feat_dynamic_real": [temp['매출총이익(요약)'].values,temp['당기순이익(요약)'].values,temp['자산총계(요약)'].values,temp['부채총계(요약)'].values]}],
    freq = "365D"
)

forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data,  # test dataset
    predictor=predictor ,  # predictor
    num_samples=1,  # number of sample paths we want for evaluation
)

In [181]:
answer_data = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익_차분'].values}],
    freq = "365D"
)

In [182]:
forecasts = list(forecast_it)
tss = list(ts_it)

In [183]:
train_entry = next(iter(forecasts))
train_entry

gluonts.model.forecast.SampleForecast(freq="365D", info=None, item_id=None, samples=numpy.array([[95128.4296875]], numpy.dtype("float32")), start_date=pandas.Timestamp("2017-12-20 00:00:00", freq="365D"))

In [184]:
train_entry.samples

array([[95128.43]], dtype=float32)

In [185]:
predict = train_entry.samples

In [186]:
predict[0][0]

95128.43

In [191]:
answer = temp['영업이익_차분'].values[-prediction_window:]

In [192]:
answer[0]

131207

In [193]:
RMSE = mean_squared_error(answer,predict[0])**0.5

In [194]:
RMSE

36078.5703125